# Modularity

In [ ]:
import numpy as np
import networkx as nx
from numba import jit, prange
import plotly.graph_objects as go
from ipywidgets import IntProgress
from IPython.display import display

## Load data

In [ ]:
filename = input('Hellinger distance file path: ')
plot_title = input('Title for the plot: ')

adj = 1.0 - np.load(filename)
np.fill_diagonal(adj, 0.0) # Don't consider diagonal edges

## Find the best modularity

In [ ]:
def find_communities(adj):
    communities = np.zeros(adj.shape[0], dtype=np.int32)

    label = 0
    pool = set()
    for idx, val in enumerate(communities):
        if val == 0:
            # Change the community
            label += 1
            communities[idx] = label
        
            # Neighbours are in the same community
            neighbours = np.nonzero(adj[idx])[0]

            # Neighbours of neighbours are in the same community
            pool |= set(neighbours)
            while pool:
                neigh_idx = pool.pop()
                neigh_val = communities[neigh_idx]

                # Don't look at previously used data
                if neigh_val == 0:
                    communities[neigh_idx] = label

                    neighbours = np.nonzero(adj[neigh_idx])[0]
                    pool |= set(neighbours)
                    
    return communities

In [ ]:
@jit(nopython=True, nogil=True, parallel=True, fastmath=True)
def compute_modularity(adj, communities):    
    n_edges_doubled = np.sum(adj)
    k_all = np.sum(adj, axis=1)
    
    out = 0.0
    for row_i_idx in prange(adj.shape[0]):
        for row_j_idx in prange(row_i_idx):
            # Compute it only for nodes of the same community
            if communities[row_i_idx] == communities[row_j_idx]:
                A_ij = adj[row_i_idx, row_j_idx]
                P_ij = (k_all[row_i_idx] * k_all[row_j_idx]) / n_edges_doubled
                
                out += A_ij - P_ij

    out /= (n_edges_doubled // 2)
    return out

In [ ]:
modularities = np.array([])
samples = np.array([])
edges = np.array([])

### Run it iteratively until you are satisfied with the plot

In [ ]:
range_start = float(input('Range (start value): '))
assert range_start > 0

range_stop = float(input('Range (stop value): '))
assert range_stop < 1

range_num = int(input('Range (number of samples): '))
assert range_num > 0

f = IntProgress(min=0, max=range_num*4)
display(f)

attempts = np.linspace(range_start, range_stop, range_num)
for attempt in attempts:
    # Loading data
    adj_bin = (adj > attempt).astype(dtype=np.int32)
    f.value += 1
    
    # Counting number of edges
    edges = np.append(edges, np.sum(adj_bin)//2)
    f.value += 1
    
    # Find communities
    communities = find_communities(adj_bin)
    f.value += 1
    
    # Compute modularity
    modularity = compute_modularity(adj_bin, communities)
    f.value += 1
    
    modularities = np.append(modularities, modularity)
    samples = np.append(samples, attempt)
    
# Sort new data before plotting it
sort_idx = samples.argsort()
samples = samples[sort_idx]
modularities = modularities[sort_idx]
edges = edges[sort_idx]

# Plot the result
fig = go.Figure(data=go.Scatter(x=samples, y=modularities, mode='lines+markers'))
fig.update_layout(title=plot_title,
                   xaxis_title='Threshold',
                   yaxis_title='Modularity')
fig.show()

fig = go.Figure(data=go.Scatter(x=samples, y=edges, mode='lines+markers'))
fig.update_layout(title=plot_title,
                   xaxis_title='Threshold',
                   yaxis_title='Number of edges')
fig.show()

## License
<small>Copyright (C) 2020 MaLGa ML4DS 

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see &lt;https://www.gnu.org/licenses/&gt;.</small>